In [ ]:
# Authenticate and install
from google.colab import auth
auth.authenticate_user()

!pip install --quiet google-cloud-bigquery pandas

#  Imports
import pandas as pd
import numpy as np
from google.cloud import bigquery

#  Initialize BigQuery client and load the table into a DataFrame
project_id = "rock-finder-project"
client = bigquery.Client(project=project_id)
table_id = "rock-finder-project.routes.mp_routes"

query = f"SELECT * FROM `{table_id}`"
df = client.query(query).to_dataframe()

#  Split Location into parts (robust to any number of ' > ' levels)
location_split = df['Location'].str.split(' > ')

#  Extract "area", "region" (island or subregion), and "state" from the last 3 parts
df['state'] = location_split.apply(lambda x: x[-1] if len(x) >= 1 else None)
df['region'] = location_split.apply(lambda x: x[-2] if len(x) >= 2 else None)
df['area'] = location_split.apply(lambda x: x[-3] if len(x) >= 3 else None)

#  first/topmost part separately:
df['top_level'] = location_split.apply(lambda x: x[0] if len(x) >= 1 else None)

# Handle invalid "Pitches" and missing "Length"
df['Pitches'] = df['Pitches'].replace(-1, np.nan)
df['Length'] = df['Length'].fillna(df['Length'].median())

# Quick sanity check
print(df[['Location', 'area', 'region', 'state']].head())
print("Missing counts:\n", df[['area','region','state','Pitches','Length']].isna().sum())

In [ ]:
df['desc'] = df['desc'].str.lower()
df['protection'] = df['protection'].str.lower()

df['desc'] = df['desc'].str.strip()
df['protection'] = df['protection'].str.strip()


df['desc'] = df['desc'].str.replace(r'\s+', ' ', regex=True)
df['protection'] = df['protection'].str.replace(r'\s+', ' ', regex=True)

df['desc'] = df['desc'].fillna('')
df['protection'] = df['protection'].fillna('')

import string
df['desc'] = df['desc'].str.translate(str.maketrans('', '', string.punctuation))
df['protection'] = df['protection'].str.translate(str.maketrans('', '', string.punctuation))
